In [1]:
import pandas as pd
from pathlib import Path
import html
import preprocessor as p
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import numpy as np
from tqdm import tqdm
from emolexHelper import pandasMultiCategorySentiment

In [2]:
BEFORE_DIR = "before_pandemic"
AFTER_DIR = "since_pandemic"

In [3]:
# load all the csv files from
#before = dict()
df = pd.DataFrame(columns=["cat", "username", "text", "date", "geo", "hashtags", "tweet_id", "mentions", "permalink", "replies", "retweets", "replies_to", "mentioned_urls"])

for file in list((Path(".") / BEFORE_DIR).glob('**/*.csv')):
    #before[file.stem] = pd.read_csv(file)
    user_df = pd.read_csv(file)
    # BUG? why are there nan text rows?
    # dropping them!
    user_df = user_df.drop(user_df[pd.isnull(user_df["text"])].index)
    user_df["cat"] = BEFORE_DIR
    user_df["username"] = file.stem
    df = df.append(user_df)
    #before[file.stem] = before[file.stem].drop(before[file.stem][pd.isnull(before[file.stem]["text"])].index)

#after = dict()
for file in list((Path(".") / AFTER_DIR).glob('**/*.csv')):
    #after[file.stem] = pd.read_csv(file)
    user_df = pd.read_csv(file)
    # BUG? why are there nan text rows?
    # dropping them!
    user_df = user_df.drop(user_df[pd.isnull(user_df["text"])].index)
    user_df["cat"] = AFTER_DIR
    user_df["username"] = file.stem
    df = df.append(user_df)
    #after[file.stem] = after[file.stem].drop(after[file.stem][pd.isnull(after[file.stem]["text"])].index)

In [4]:
def decodeAndClean(s:str):
    out = html.unescape(s)
    out = p.clean(out)
    return out

In [5]:
import nltk
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')

def preprocess(text):
    text = text.lower()
    
    text_p = "".join([char for char in text if char not in string.punctuation])
    
    words = word_tokenize(text_p)
    
    stop_words = stopwords.words('english')
    filtered_words = [word for word in words if word not in stop_words]
    
    porter = PorterStemmer()
    stemmed = [porter.stem(word) for word in filtered_words]
    
    return " ".join(filtered_words) #convert stemmed words to 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maxim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
df

,cat,username,text,date,geo,hashtags,tweet_id,mentions,permalink,replies,retweets,replies_to,mentioned_urls
0,before_pandemic,1baldchick,big wet smooshy kisses to you!!! i love u dear!!,2020-02-28 19:46:02+00:00,NaN,NaN,1233478483939971072,NaN,https://twitter.com/1baldchick/status/12334784...,0,0,amarcadia,NaN
1,before_pandemic,1baldchick,list of meds and check it for drug interaction...,2020-02-28 19:45:26+00:00,NaN,NaN,1233478333511479299,NaN,https://twitter.com/1baldchick/status/12334783...,1,0,1baldchick,NaN
2,before_pandemic,1baldchick,gab has loads of side effects. i always try to...,2020-02-28 19:44:18+00:00,NaN,NaN,1233478047212408834,NaN,https://twitter.com/1baldchick/status/12334780...,1,0,PajamaDaze,http://drugs.com
3,before_pandemic,1baldchick,happy friday to u good sir!! and to your littl...,2020-02-28 19:42:32+00:00,NaN,NaN,1233477603866050560,NaN,https://twitter.com/1baldchick/status/12334776...,0,0,ChronicPainDad,NaN
4,before_pandemic,1baldchick,this song says it all!,2020-02-28 08:34:04+00:00,NaN,NaN,1233309379497283584,NaN,https://twitter.com/1baldchick/status/12333093...,0,0,halsey,https://twitter.com/halsey/status/123326171381...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,since_pandemic,_tomdalton,total bullshit narrative.,2020-03-07 03:27:53+00:00,NaN,NaN,1236131428892651520,NaN,https://twitter.com/_tomdalton/status/12361314...,0,0,germanrlopez,NaN
271,since_pandemic,_tomdalton,doctors are not always right either and many w...,2020-03-05 23:36:05+00:00,NaN,NaN,1235710707993374721,NaN,https://twitter.com/_tomdalton/status/12357107...,0,1,StSenka,NaN
272,since_pandemic,_tomdalton,yes x1000,2020-03-05 18:57:13+00:00,NaN,NaN,1235640525740552192,NaN,https://twitter.com/_tomdalton/status/12356405...,0,1,TruthPharm,https://twitter.com/TruthPharm/status/12355996...
273,since_pandemic,_tomdalton,very important thread. already hearing the anx...,2020-03-05 18:55:54+00:00,NaN,NaN,1235640197926264832,@MorganGodvin,https://twitter.com/_tomdalton/status/12356401...,1,1,MorganGodvin,https://twitter.com/MorganGodvin/status/123561...


In [7]:
df["text"] = np.vectorize(decodeAndClean)(df["text"])

In [8]:
df["text_nlp"] = np.vectorize(preprocess)(df["text"])

In [9]:
df["date"] = list(map(lambda s: s[:-3] + s[-2:], df["date"]))
df = df.drop(df[df["date"]=="dte"].index) # BUG? For some reason some date colums contain the value "dte"
df["date"] = pd.to_datetime(df["date"])
df = df.reset_index()

<ipython-input-9-85ea215eeb4a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["date"] = pd.to_datetime(df["date"])


In [10]:
grouped = df.groupby(pd.Grouper(key="date", freq='5d'))

In [11]:
from tqdm import tqdm

In [12]:
path = (Path(".") / "export")
for name, group in grouped: #tqdm(list(grouped)):
    fname = str(name).split()[0].replace("-", "_")
    text = list(group["text"].values)
    fileHandle = open(str((path / fname).absolute()) + ".txt", "w")
    fileHandle.write("\n".join(text))
    # fileHandle.writelines(text)
    fileHandle.close()
    print(f"Processing {fname} ({len(text)} lines)")
    # print(fname)
    # print(text)
    # print(len(text))
    # break

Processing 2019_10_01 (2714 lines)
Processing 2019_10_06 (2947 lines)
Processing 2019_10_11 (2926 lines)
Processing 2019_10_16 (2652 lines)
Processing 2019_10_21 (2580 lines)
Processing 2019_10_26 (2608 lines)
Processing 2019_10_31 (2810 lines)
Processing 2019_11_05 (2768 lines)
Processing 2019_11_10 (2762 lines)
Processing 2019_11_15 (2810 lines)
Processing 2019_11_20 (3267 lines)
Processing 2019_11_25 (3099 lines)
Processing 2019_11_30 (3202 lines)
Processing 2019_12_05 (3398 lines)
Processing 2019_12_10 (3231 lines)
Processing 2019_12_15 (3090 lines)
Processing 2019_12_20 (2964 lines)
Processing 2019_12_25 (2994 lines)
Processing 2019_12_30 (2594 lines)
Processing 2020_01_04 (2771 lines)
Processing 2020_01_09 (3709 lines)
Processing 2020_01_14 (3453 lines)
Processing 2020_01_19 (3285 lines)
Processing 2020_01_24 (3899 lines)
Processing 2020_01_29 (4122 lines)
Processing 2020_02_03 (3645 lines)
Processing 2020_02_08 (4170 lines)
Processing 2020_02_13 (3790 lines)
Processing 2020_02_1